In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

df_bike_station = pd.read_csv('../data/station/서울시 따릉이대여소 마스터 정보.csv', encoding='cp949')
df_bike_station = df_bike_station.loc[df_bike_station.loc[:,'위도']!=0]
df_bike_station = df_bike_station.loc[df_bike_station.loc[:,'경도']!=0]

In [2]:
list_point = [Point(xy) for xy in zip(df_bike_station['경도'], df_bike_station['위도'])]

In [3]:
gdf_bike_station = gpd.GeoDataFrame(df_bike_station, geometry=list_point, crs=4326)

In [4]:
gdf_bike_station

,대여소_ID,주소1,주소2,위도,경도,geometry
2,ST-997,서울특별시 양천구 목동중앙로 49,목동3단지 시내버스정류장,37.534390,126.869598,POINT (126.86960 37.53439)
3,ST-996,서울특별시 양천구 남부순환로88길5-16,양강중학교앞 교차로,37.524334,126.850548,POINT (126.85055 37.52433)
4,ST-995,서울특별시 양천구 중앙로 153 공중화장실,NaN,37.510597,126.857323,POINT (126.85732 37.51060)
5,ST-994,서울특별시 양천구 목동서로161,SBS방송국,37.529163,126.872749,POINT (126.87275 37.52916)
6,ST-993,서울특별시 양천구 신월로 342-1 구두수선대19,NaN,37.521511,126.857384,POINT (126.85738 37.52151)
...,...,...,...,...,...,...
3297,ST-1003,서울특별시 양천구 화곡로 59,신월동 이마트,37.539551,126.828300,POINT (126.82830 37.53955)
3298,ST-1002,서울특별시 양천구 목동동로 316-6,서울시 도로환경관리센터,37.529900,126.876541,POINT (126.87654 37.52990)
3300,ST-1000,서울특별시 양천구 신정동 236,서부식자재마트 건너편,37.510380,126.866798,POINT (126.86680 37.51038)
3301,ST-100,서울특별시 광진구 아차산로 262,더샵스타시티 C동 앞,37.536667,127.073593,POINT (127.07359 37.53667)


In [5]:
df_subway_station = pd.read_csv('../data/subway_station/서울시 역사마스터 정보.csv', encoding='cp949')
list_point = [Point(xy) for xy in zip(df_subway_station['경도'], df_subway_station['위도'])]


In [6]:
gdf_subway_station = gpd.GeoDataFrame(df_subway_station, geometry=list_point, crs=4326)
gdf_subway_station.to_file('../data/subway_station/subway_station.geojson', driver='GeoJSON')

In [7]:
gdf_bike_station = gdf_bike_station.to_crs(5174)
gdf_subway_station = gdf_subway_station.to_crs(5174)
gdf_bike_station = gdf_bike_station.sjoin_nearest(gdf_subway_station.loc[:,['geometry']], how='left', distance_col='dist_to_subway').drop('index_right', axis=1)

In [8]:
gdf_bike_station = gdf_bike_station.rename(columns={'대여소_ID':'station_id',
                                          '주소1':'address',
                                          '주소2':'address_detail',
                                          '위도':'latitude',
                                          '경도':'longitude'})

In [9]:
gdf_bike_station

,station_id,address,address_detail,latitude,longitude,geometry,dist_to_subway
2,ST-997,서울특별시 양천구 목동중앙로 49,목동3단지 시내버스정류장,37.534390,126.869598,POINT (188404.676 448023.915),1011.885895
3,ST-996,서울특별시 양천구 남부순환로88길5-16,양강중학교앞 교차로,37.524334,126.850548,POINT (186719.138 446910.345),504.224092
4,ST-995,서울특별시 양천구 중앙로 153 공중화장실,NaN,37.510597,126.857323,POINT (187315.694 445384.759),777.296433
5,ST-994,서울특별시 양천구 목동서로161,SBS방송국,37.529163,126.872749,POINT (188682.385 447443.391),560.822569
6,ST-993,서울특별시 양천구 신월로 342-1 구두수선대19,NaN,37.521511,126.857384,POINT (187322.943 446596.077),401.021544
...,...,...,...,...,...,...,...
3297,ST-1003,서울특별시 양천구 화곡로 59,신월동 이마트,37.539551,126.828300,POINT (184755.579 448602.649),1096.610445
3298,ST-1002,서울특별시 양천구 목동동로 316-6,서울시 도로환경관리센터,37.529900,126.876541,POINT (189017.675 447524.738),611.711570
3300,ST-1000,서울특별시 양천구 신정동 236,서부식자재마트 건너편,37.510380,126.866798,POINT (188153.380 445359.433),240.117803
3301,ST-100,서울특별시 광진구 아차산로 262,더샵스타시티 C동 앞,37.536667,127.073593,POINT (206434.705 448270.900),510.952220


In [10]:
gdf_bike_station.to_file('../data/station/bike_station.geojson', driver='GeoJSON')

In [12]:
gdf_subway_station.rename(columns={'역사명':'station_name','위도':'latitude','경도':'longitude'}).loc[:,['station_name','latitude','longitude']].to_json('../data/subway_station/subway_station.json', orient='records', force_ascii=False)

In [26]:
gdf_subway_station_agg = gdf_subway_station.groupby('역사명', as_index=False).mean().drop('역사_ID', axis=1)
gdf_subway_station_agg = gdf_subway_station_agg.merge(gdf_subway_station.groupby('역사명', as_index=False).size().rename(columns={'size':'count'}), on='역사명')
gdf_subway_station_agg.rename(columns={'역사명':'station_name','위도':'latitude','경도':'longitude'}).loc[:,['station_name','latitude','longitude','count']].to_json('../data/subway_station/subway_station.json', orient='records')

C:\Users\seung\AppData\Local\Temp\ipykernel_1112\4167862034.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  gdf_subway_station_agg = gdf_subway_station.groupby('역사명', as_index=False).mean().drop('역사_ID', axis=1)


In [27]:
gdf_subway_station_agg

,역사명,위도,경도,count
0,4.19민주묘지,37.649502,127.013684,1
1,가능,37.748577,127.044213,1
2,가락시장,37.492566,127.118077,2
3,가산디지털단지,37.480959,126.882619,2
4,가양,37.561391,126.854456,1
...,...,...,...,...
643,회현(남대문시장),37.558514,126.978246,1
644,효자,37.754025,127.076902,1
645,효창공원앞,37.538906,126.961797,2
646,흑석(중앙대입구),37.508770,126.963708,1
